# Výjimky a chyby
Chyby v Pythonu můžeme rozlišit na *syntaktické chyby* a *chyby za běhu* (*run-time*). Syntaktické chyby jsou způsobeny např. nespárovanými závorkami, špatným odsazením apod. Program se syntaktickými chybami nelze vůbec spustit. Častější jsou run-time chyby, které vznikají nesprávným použitím nějaké funkce, chybějícími daty apod. Jelikož je Python interpretovaný jazyk, většina chyb se ukáže až za běhu, na rozdíl od kompilovaných jazyků, kde se mnoho chyb objeví při kompilaci.
<!-- TEASER_END -->

## Výjimky

Výjimka (*exception*) je vyhozena ve chvíli, kdy dojde k chybě (respektivě ve chvíli kdy autor funkce kterou používáme považuje naše užití za chybné :-)). Pokud tuto výjimku nezachytíme (viz dále), běh programu se přeruší. Např. dělení nulou skončí výjimkou `ZeroDivisionError`:

### Chytáme výjimky
Pokud nechceme, aby běh programu skončil ve chvíli výjimky, můžeme použít `try` - `except` blok. Ten funguje tak, že rizikovou část kód umístíme do `try` bloku, do `except` bloku pak umístíme instrukce pro případ chyby (výjimky).

In [3]:
# ukázka try/except

def deleni(a, b):
    return a / b

def deleni_upravene(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return "Nelze dělit nulou!"
    
print(deleni_upravene(1, 0))
print(deleni_upravene(1, 2))
print(deleni(1, 0))

Nelze dělit nulou!
0.5


ZeroDivisionError: division by zero

`except` lze také použít bez specifikace typu výjimky, v tom případě se zachytí všechny výjimky. Toto ale není dobrá praxe, protože vlastně nevíme jaký typ chyby "obcházíme".

Kompletní try-except blok může ještě obsahovat `else` a `finally` bloky, viz [dokumentace](http://docs.python.org/3/reference/compound_stmts.html#try). `finally` se hodí zejména pro "úklid", např. zavření souboru apod.

In [4]:
# kopletní try/except/else/finally
a = 1
b = 0
try:
    c = a / b
except:
    print("Nastala nějaká chyba, nestarám se o to jaká.")
else:
    print("Všechno v pořádku.")
finally:
    print("Toto se vždy provede.")

Nastala nějaká chyba, nestarám se o to jaká.
Toto se vždy provede.


Pomocí modulu traceback si můžeme nechat vypsat podrobnější informace jak k vyjímce došlo, to se může hodit při ladění.

In [5]:
import traceback

a = 1
b = 0
try:
    c = a / b
except:
    print("Nastala nějaká chyba, nestarám se o to jaká.")
    traceback.print_exc() # dá se nastavit na výstup do souboru
else:
    print("Všechno v pořádku.")
finally:
    print("Toto se vždy provede.")

print("A stale jedeme dál.")


Nastala nějaká chyba, nestarám se o to jaká.
Toto se vždy provede.
A stale jedeme dál.


Traceback (most recent call last):
  File "/tmp/ipykernel_4120/1295279126.py", line 6, in <module>
    c = a / b
ZeroDivisionError: division by zero


### Vytváříme vlastní výjimky
Výjimku můžeme samozřejmě vyhodit i v našem kódu pomocí klíčového slova `raise`. Pokud bychom chtěli např. kontrolovat vstup nějaké funkce, uděláme to takto:

In [6]:
# ukázka raise
def zaplatit(cena):
    if cena > 100:
        raise ValueError("Cena je moc vysoká!")
    else:
        print("Zaplatil jsem", cena, "Kč.")
        
zaplatit(50)
zaplatit(150)

Zaplatil jsem 50 Kč.


ValueError: Cena je moc vysoká!

# Generátory a iterátory
Abychom se v tomto tématu lépe zorientovali, začneme s rozčleněním. Budeme používat tyto tři termíny:
- `iterable` (iterovatelný objekt) - objekt, který umí vracet své prvky jeden po druhém
    - kontejnery (list, slovník, ...)
    - řetězec
    - range
    - objekt typu stream (např. `file`)
- `iterator` (iterátor) - objekt, který umí iterovat (implementuje tzv. protokol iterátoru). Lze vytvořít z iterovatelného objektu pomocí funkce `iter()`.
    - metoda `__iter__` vrací samotný objekt
    - metoda `__next__` vrací další prvek (na konci vyhodí výjimku `StopIteration`)
- `generator` (generátor) - je typ iterátoru. Má dvě varianty:
    - generátor funkce - funkce, která vrací výstupy postupně pomocí `yield` místo `return`
    - generátor výraz - výraz využívající syntaxe: **(**`výraz` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**)**


  Pěkné vysvětlení lze najít i zde: [Iterables vs. Iterators vs. Generators](http://nvie.com/posts/iterators-vs-generators/)

In [7]:
# ukázka iterable objektů
muj_list = [1, 2, 3, 4, 5]
muj_string = "ahoj"
muj_double = 1.5
print(iter(muj_list))
print(iter(muj_string))
# print(iter(muj_double))  # tohle skončí chybou

In [10]:
# ukázka iterátorů
muj_list = [1, 2, 3, 4, 5]
muj_iterator = iter(muj_list)
print(muj_iterator.__iter__())
print(muj_iterator.__next__())
print(next(muj_iterator))
print(next(muj_iterator))
print(muj_iterator.__next__())
print(next(muj_iterator))
print(next(muj_iterator))

1
2
3
4
5


StopIteration: 

In [11]:
# generator funkce
def generator1():
    yield "Ahoj"
    yield "světe"
    yield "jak"
    yield "se"
    yield "máš?"


def generator2(start, konec):
    for i in range(start, konec):
        print("A tohle je další iterace!")
        yield i

# ukázka použití generátoru
gen1 = generator1()
print(gen1)
print(next(gen1))
print(next(gen1))
print(next(gen1))
print(next(gen1))
print(next(gen1))
try:
    print(next(gen1))
except StopIteration:
    print(f"Generátor {gen1} vyčerpal všechny hodnoty.")
    
gen2 = generator2(1, 6)
print(gen2)
print(next(gen2))
print(next(gen2))
print(next(gen2))
print(next(gen2))
print(next(gen2))
try:
    print(next(gen2))
except StopIteration:
    print(f"Generátor {gen2} vyčerpal všechny hodnoty.")


<generator object generator1 at 0x7fed103619a0>
Ahoj
světe
jak
se
máš?
Generátor <generator object generator1 at 0x7fed103619a0> vyčerpal všechny hodnoty.
<generator object generator2 at 0x7fed10361c40>
A tohle je další iterace!
1
A tohle je další iterace!
2
A tohle je další iterace!
3
A tohle je další iterace!
4
A tohle je další iterace!
5
Generátor <generator object generator2 at 0x7fed10361c40> vyčerpal všechny hodnoty.


In [12]:
# generatorový výraz
gen3 = (i for i in range(1, 6))
print(gen3)
print(next(gen3))
print(next(gen3))
print(next(gen3))
print(next(gen3))
print(next(gen3))
try:
    print(next(gen3))
except StopIteration:
    print(f"Generátor {gen3} vyčerpal všechny hodnoty.")

<generator object <genexpr> at 0x7fed10362340>
1
2
3
4
5
Generátor <generator object <genexpr> at 0x7fed10362340> vyčerpal všechny hodnoty.


In [13]:
# generatorový výraz s podmínkou
gen4 = (i for i in range(1, 8) if i % 2 == 0)
print(gen4)
print(next(gen4))
print(next(gen4))
print(next(gen4))
try:
    print(next(gen4))
except StopIteration:
    print(f"Generátor {gen4} vyčerpal všechny hodnoty.")

<generator object <genexpr> at 0x7fed103631b0>
2
4
6
Generátor <generator object <genexpr> at 0x7fed103631b0> vyčerpal všechny hodnoty.


# List/Set/Dict comprehensions
List/Set/Dict comprehensions jsou způsob, jak vytvořit seznam, množinu nebo slovník pomocí jednoho řádku kódu. Všechny tyto konstrukce jsou velmi podobné jako generator výrazy.
- List comprehension - vytvoří seznam, syntaxe: **[**`výraz` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**]**
- Set comprehension - vytvoří množinu, syntaxe: **{**`výraz` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**}**
- Dict comprehension - vytvoří slovník, syntaxe: **{**`klíč` **:** `hodnota` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**}**

In [14]:
# list comprehension
print([i for i in range(1, 6)])
print([i for i in range(1, 6) if i % 2 == 0])
print([i + 1 for i in range(1, 6) if i % 2 == 0])
print([(i, i**2, i**3) for i in range(1, 6)])

[1, 2, 3, 4, 5]
[2, 4]
[3, 5]
[(1, 1, 1), (2, 4, 8), (3, 9, 27), (4, 16, 64), (5, 25, 125)]


In [15]:
# set comprehension
print({i for i in range(1, 6)})
print({i for i in range(1, 6) if i % 2 == 0})
print({i + 1 for i in range(1, 6) if i % 2 == 0})
print({(i, i**2, i**3) for i in range(1, 6)})

{1, 2, 3, 4, 5}
{2, 4}
{3, 5}
{(2, 4, 8), (4, 16, 64), (3, 9, 27), (5, 25, 125), (1, 1, 1)}


In [16]:
# dict comprehension
print({i: i**2 for i in range(1, 6)})
print({i: i**2 for i in range(1, 6) if i % 2 == 0})
print({i + 1: i**2 for i in range(1, 6) if i % 2 == 0})
print({i: (i**2, i**3) for i in range(1, 6)})

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25}
{2: 4, 4: 16}
{3: 4, 5: 16}
{1: (1, 1), 2: (4, 8), 3: (9, 27), 4: (16, 64), 5: (25, 125)}
